In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')

In [4]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [6]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [7]:
pd.read_sql(
    """
    select 
    count (*) filter (where CRIM=0.0) as CRIM,
    count(*) filter (where zn=0.0) as ZN,
    count (*) filter (where INDUS=0.0) as INDUS,
    count (*) filter (where CHAS=0.0) as CHAS,
    count (*) filter (where NOX=0.0) as NOX from boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,372,0,471,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [8]:
pd.read_sql(
    """
    select count(distinct CRIM) filter (where CRIM!=0.0) as CRIM,
    count (distinct ZN) filter (where ZN!=0.0) as ZN,
    count (distinct INDUS) filter (where INDUS!=0.0) as INDUS,
    count (distinct CHAS) filter (where CHAS!=0.0) as CHAS,
    count (distinct NOX) filter (where NOX!=0.0) as NOX
    from boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,25,76,1,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [58]:
pd.read_sql(
    """
    with medians as (
    select
    nth_value(CRIM, (select count(*)/2 from boston)) over (order by CRIM) as CRIM_MEDIAN,
    min(CRIM) over () as MIN_CRIM,
    nth_value(ZN, (select count(*)/2 from boston)) over (order by ZN) as ZN_MEDIAN,
    min(ZN) over () as MIN_ZN,
    nth_value(INDUS, (select count(*)/2 from boston)) over (order by INDUS) as INDUS_MEDIAN,
    min(INDUS) over () as MIN_INDUS,
    nth_value(CHAS, (select count(*)/2 from boston)) over (order by CHAS) as CHAS_MEDIAN,
    min(CHAS) over () as MIN_CHAS,
    nth_value(NOX, (select count(*)/2 from boston)) over (order by NOX) as NOX_MEDIAN,
    min(NOX) over () as MIN_NOX
    from boston  limit 1 offset (select count(*) from boston) /2
    )
    select
     iif(CRIM_MEDIAN=MIN_CRIM, 'CRIM', ' ') ||
     iif(ZN_MEDIAN=MIN_ZN, 'ZN', ' ') ||
     iif(INDUS_MEDIAN=MIN_INDUS, 'INDUS', ' ') ||
     iif(CHAS_MEDIAN=MIN_CHAS, 'CHAS', ' ') ||
     iif(NOX_MEDIAN=MIN_NOX, 'NOX', ' ') as task_result
    from medians
    """,
    con,
)

,task_result
0,ZN CHAS


Согласно wiki: Если распределение имеет разрывы, то медиана может совпадать с минимальным или максимальным (крайним) возможным значением случайной величины, что противоречит «геометрическому» пониманию этого термина.
[Расчет медианы по stackoverflow](https://stackoverflow.com/questions/15763965/how-can-i-calculate-the-median-of-values-in-sqlite)

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [157]:
pd.read_sql(
    """
    with top_houses as (
        select avg(RM) as avg_rm_top_price from boston where medv=(select max(medv) from boston)
    ), min_houses_ntiled as (
        select
            ntile(25) over (order by medv) as ntile_25,
            ntile(50) over (order by medv) as ntile_50,
            ntile(100) over (order by medv) as ntile_100,
            ntile(200) over (order by medv) as ntile_200,
            ntile(300) over (order by medv) as ntile_300,
        RM
        from boston order by medv
    )
    select
        (select max(avg_rm_top_price) from top_houses) - avg(RM) filter (where ntile_25=1) as diff_25,
        (select max(avg_rm_top_price) from top_houses) - avg(RM) filter (where ntile_50=1) as diff_50,
        (select max(avg_rm_top_price) from top_houses) - avg(RM) filter (where ntile_100=1) as diff_100,
        (select max(avg_rm_top_price) from top_houses) - avg(RM) filter (where ntile_200=1) as diff_200,
        (select max(avg_rm_top_price) from top_houses) - avg(RM) filter (where ntile_300=1) as diff_300

    from min_houses_ntiled
    """,
    con,
)

,diff_25,diff_50,diff_100,diff_200,diff_300
0,1.199366,1.78,1.999333,1.776333,1.916


В целом по разнице в средних на 50, 100, 200, 300 разница прослеживается, в то же время - минимальные 25 при сравнении с максимальными 25 -
существенной разницы не показывают. Полагаю можно считать, что в дорогих домах на avg(diff_25, diff_50, diff_100, diff_200, diff_300) комнат в среднем больше, чем в дешевых.

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [174]:
pd.read_sql(
    """
    with ranks as (
    select LSTAT, MEDV ,RANK() over (order by MEDV) as RANK from boston order by MEDV desc
    ) select * from ranks where MEDV = (select max(MEDV) from boston)
    """,
    con
)

,LSTAT,MEDV,RANK
0,1.73,50.0,491
1,1.92,50.0,491
2,3.32,50.0,491
3,3.70,50.0,491
4,4.45,50.0,491
5,2.97,50.0,491
6,2.88,50.0,491
7,4.63,50.0,491
8,5.12,50.0,491
9,7.44,50.0,491


In [148]:
pd.read_sql(
    """
    """,
    con,
)

,avg(RM)
0,6.284634


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [ ]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)